#### Tests fitting 1D functions with the NEAT algorithm - Fixed Activations

In [ ]:
from matplotlib import pyplot as plt
import autograd.numpy as np
import random
import sys
import os

# Add src directory to path (where 'evograd' package is located)
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
root_dir = os.path.abspath(os.path.join(notebook_dir, '../..'))
src_dir = os.path.join(root_dir, 'src')
regression_1D_dir = os.path.dirname(os.path.abspath('__file__'))  # Current regression_1D directory
sys.path.insert(0, src_dir)
sys.path.insert(0, regression_1D_dir)

from evograd.activations     import sigmoid_activation
from evograd.run             import Config
from trial_regression1D      import Trial_Regression1D    , Experiment_Regression1D
from trial_regression1D_grad import Trial_Regression1DGrad, Experiment_Regression1DGrad

In [ ]:
# LOAD CONFIGURATION DATA
config = Config("config_regression1D.ini")
config.fitness_threshold      = 9.98   # maximum fitness is 10.0
config.population_size        = 100
config.max_number_generations = 300
config.enable_gradient        = True
config.lamarckian_evolution   = True
NETWORK_TYPE = "autograd"

In [ ]:
# Define the function we want to approximate.
def foo(x):
    x1 = sigmoid_activation(x)
    return sigmoid_activation(x1)

# Define the range on which we approximate the function.
X_MIN = -1.0
X_MAX = +0.5

In [ ]:
# Run one trial, defined as one attempt to find a network that approximates the given function.
# Display progress after each generation and visualize final fittest network.

# Set both random seeds for reproducibility
if True:
    random.seed(27)      
    np.random.seed(27)   

if config.enable_gradient:
    config.gradient_steps = 500
    trial = Trial_Regression1DGrad(config, foo, X_MIN, X_MAX)
    trial.run(num_jobs=-1)  
else:
    trial = Trial_Regression1D(config, NETWORK_TYPE, foo, X_MIN, X_MAX)
    trial.run(num_jobs=1)   # no benefit in parallelizing the (very simple) fitness evalution

In [ ]:
# Display the results.
champion = trial._population.get_fittest_individual()

if config.enable_gradient:
    Xs, Ys = trial._get_training_data()
else:
    Xs = np.linspace(X_MIN, X_MAX, 100)
    Ys = np.array([foo(x) for x in Xs])
    
Os = np.array([champion._network.forward_pass([x])[0] for x in Xs])

plt.plot(Xs, Ys, label="target")
plt.plot(Xs, Os, label="fitted")
plt.grid(linewidth=0.25)
plt.legend()

In [ ]:
# Run an experiment, defined as a collection of multiple trials.
# Display summary of each trial and aggregated statistics.
NUM_TRIALS = 50

if config.enable_gradient:
    experiment = Experiment_Regression1DGrad(NUM_TRIALS, config, function=foo, x_min=X_MIN, x_max=X_MAX)
else:
    experiment = Experiment_Regression1D(NUM_TRIALS, config, NETWORK_TYPE, function=foo, x_min=X_MIN, x_max=X_MAX)
experiment.run(num_jobs_trials=-1, num_jobs_fitness=1)